# Udacity Capstone Project

## EEG signal Based Eye State Classification

### Loading the files in Google colab

In [17]:
# Loading the dataset

fname = "EEG Eye State.txt"
with open(fname) as f:
    content = f.readlines()

content = [x.strip() for x in content] 
content = [x.split(",") for x in content]

In [18]:
# Converting list to numpy array

import numpy as np

content = np.array(content, dtype = 'float32')

In [19]:
# Shuffling the dataset

import random
random.shuffle(content)

In [20]:
# Storing results of algorithms
score_p = []

### Creating X and Y

In [21]:
x = content[:, :-1]
y = np.array(content[:, -1], dtype = 'int32')

In [22]:
x[0]

array([4329.23, 4009.23, 4289.23, 4148.21, 4350.26, 4586.15, 4096.92,
       4641.03, 4222.05, 4238.46, 4211.28, 4280.51, 4635.9 , 4393.85],
      dtype=float32)

In [23]:
y[0]

0

# Statistical Features approach

## Data Preparation

In [24]:
import pandas as pd
import numpy as np
import scipy

In [25]:
X_columns = ['mean', 'standard deviation', 'kurt', 'skewness']
Y_columns = ['label']

In [26]:
X = pd.DataFrame(columns = X_columns)
Y = pd.DataFrame(columns = Y_columns)

In [27]:
for i in range(len(x)):
  X.loc[i] = np.array([np.mean(x[i]), np.std(x[i]), scipy.stats.kurtosis(x[i]), scipy.stats.skew(x[i])])
  Y.loc[i] = y[i]

AttributeError: module 'scipy' has no attribute 'stats'

In [28]:
X.head(n=20)

,mean,standard deviation,kurt,skewness


In [29]:
Y.head()

,label


In [30]:
from sklearn.model_selection import train_test_split

# Split the 'features' and 'income' data into training and testing sets
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, 
                                                    y, 
                                                    test_size = 0.2, 
                                                    random_state = 0)



ValueError: Found input variables with inconsistent numbers of samples: [0, 14980]

## Training on SVM

In [31]:
from sklearn.svm import SVC
clf = SVC()
clf.fit(X_train1, y_train1) 

NameError: name 'X_train1' is not defined

In [32]:
predicted = clf.predict(X_test1)

NameError: name 'X_test1' is not defined

In [33]:
from sklearn import metrics

### Performance

In [34]:
print("Accuracy = {}\nPrecision = {}\nRecall = {}\nF1 Score = {}".format(metrics.accuracy_score(y_test1, predicted), metrics.precision_score(y_test1, predicted),metrics.recall_score(y_test1, predicted),metrics.f1_score(y_test1, predicted)))

score_p.append([metrics.accuracy_score(y_test1, predicted), metrics.precision_score(y_test1, predicted),metrics.recall_score(y_test1, predicted),metrics.f1_score(y_test1, predicted)]) 


NameError: name 'y_test1' is not defined

## Training on Logistic Regression

In [35]:
from sklearn.linear_model import LogisticRegression

In [36]:
clf_D = LogisticRegression()
clf_D.fit(X_train1, y_train1)


NameError: name 'X_train1' is not defined

In [37]:
predict = clf_D.predict(X_test1)


NameError: name 'X_test1' is not defined

### Performance

In [ ]:
print("Accuracy = {}\nPrecision = {}\nRecall = {}\nF1 Score = {}".format(metrics.accuracy_score(y_test1, predict), metrics.precision_score(y_test1, predict),metrics.recall_score(y_test1, predict),metrics.f1_score(y_test1, predict)))
score_p.append([metrics.accuracy_score(y_test1, predict), metrics.precision_score(y_test1, predict),metrics.recall_score(y_test1, predict),metrics.f1_score(y_test1, predict)])

## Second Approach Directly use 14 values of EEG data and use it for prediction 

### Normalization of data

Normalization of data is required before using it on Nueral Networks

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
scaler.fit(x)
x_new = scaler.transform(x)

In [ ]:
data_mean = x.mean()
data_std = x.std()
x = (x - data_mean)/data_std

In [ ]:
(x_train, x_test) = x[:11000], x[11000:]
(y_train, y_test) = y[:11000], y[11000:]

## Training on Neural Networks

### Creating Model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
model = Sequential()
model.add(Dense(512, activation = 'relu', input_shape = (x.shape[1], )))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

### Training

In [ ]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath = 'MLP.weights.best.hdf5', verbose = 1, save_best_only = True)
hist = model.fit(x_train, y_train, epochs = 100, batch_size=256, validation_split = 0.1, callbacks = [checkpointer], verbose = 2, shuffle = True)

In [ ]:
score = model.evaluate(x_test, y_test, verbose=1)
print("Accuracy: ", score[1])

In [ ]:
predict2 = [1 if a>0.5 else 0 for a in model.predict(x_test)] 

### Performance

In [ ]:
print("Accuracy = {}\nPrecision = {}\nRecall = {}\nF1 Score = {}".format(metrics.accuracy_score(y_test, predict2), metrics.precision_score(y_test, predict2),metrics.recall_score(y_test, predict2),metrics.f1_score(y_test, predict2)))
#score_p.append([metrics.accuracy_score(y_test, predict2), metrics.precision_score(y_test, predict2),metrics.recall_score(y_test, predict2),metrics.f1_score(y_test, predict2)])

# Improved Neural Network (Tuning Parameters)

In [ ]:
model2 = Sequential()
model2.add(Dense(1000, activation = 'relu', input_shape = (x.shape[1], )))
model2.add(Dropout(0.2))
model2.add(Dense(1000, activation = 'relu'))
model2.add(Dropout(0.2))
model2.add(Dense(1000, activation = 'relu'))
model2.add(Dropout(0.2))
model2.add(Dense(1, activation = 'sigmoid'))
model2.summary()

In [ ]:
model2.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath = 'MLP_new.weights.best.hdf5', verbose = 1, save_best_only = True)
hist1 = model2.fit( x_train, y_train, epochs = 200, batch_size=512, validation_split = 0.1, callbacks = [checkpointer], verbose = 2, shuffle = True )

In [ ]:
score = model2.evaluate(x_test, y_test, verbose=1)
print("Accuracy: ", score[1])

In [ ]:
predict3 = [1 if a>=0.5 else 0 for a in model2.predict(x_test)] 

In [ ]:
print("Accuracy = {}\nPrecision = {}\nRecall = {}\nF1 Score = {}".format(metrics.accuracy_score(y_test, predict3), metrics.precision_score(y_test, predict3),metrics.recall_score(y_test, predict3),metrics.f1_score(y_test, predict3)))
score_p.append([metrics.accuracy_score(y_test, predict3), metrics.precision_score(y_test, predict3),metrics.recall_score(y_test, predict3),metrics.f1_score(y_test, predict3)])

In [ ]:
import matplotlib.pyplot as plt

# Training on  LSTM

### Creating 3D array dataset

In [ ]:
X_train = np.asarray(np.reshape(x_train, (x_train.shape[0], 14, 1)))
X_test = np.asarray(np.reshape(x_test, (x_test.shape[0], 14, 1)))

In [ ]:
from keras.layers import Dense, Dropout, CuDNNLSTM, Embedding, LSTM
from keras.models import Sequential

In [ ]:
model3 = Sequential()
model3.add(LSTM(256,input_shape=(14, 1), return_sequences=True))
model3.add(LSTM(256))
model3.add(Dense(1, activation='sigmoid'))

In [ ]:
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath = 'LSTM.weights.best.hdf5', verbose = 1, save_best_only = True)
hist = model3.fit(X_train, y_train, epochs = 100, batch_size=256, validation_split = 0.1, callbacks = [checkpointer], verbose = 2, shuffle = True)

In [ ]:
score = model3.evaluate(X_test, y_test, verbose=1)
print("Accuracy: ", score[1])

In [ ]:
predict4 =  [1 if a>0.5 else 0 for a in model3.predict(X_test)]

### Performance

In [ ]:
print("Accuracy = {}\nPrecision = {}\nRecall = {}\nF1 Score = {}".format(metrics.accuracy_score(y_test, predict4), metrics.precision_score(y_test, predict4),metrics.recall_score(y_test, predict4),metrics.f1_score(y_test, predict4)))


# Improved LSTM (Tuning Parameter)

In [ ]:
model4 = Sequential()
model4.add(CuDNNLSTM(units=512, input_shape=(14, 1), return_sequences=True))
model4.add(CuDNNLSTM(units=512))
model4.add(Dense(1, activation='sigmoid'))
model4.summary()

In [ ]:
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model4.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath = 'LSTM_new.weights.best.hdf5', verbose = 1, save_best_only = True)
hist = model4.fit(X_train, y_train, epochs = 300, batch_size= 512, validation_split = 0.1, callbacks = [checkpointer], verbose = 2, shuffle = True)

In [ ]:
score = model4.evaluate(X_test, y_test, verbose=1)
print("Accuracy: ", score[1])

In [ ]:
predict5 =  [1 if a>0.5 else 0 for a in model4.predict(X_test)]

In [ ]:
print("Accuracy = {}\nPrecision = {}\nRecall = {}\nF1 Score = {}".format(metrics.accuracy_score(y_test, predict5), metrics.precision_score(y_test, predict5),metrics.recall_score(y_test, predict5),metrics.f1_score(y_test, predict5)))
score_p.append([metrics.accuracy_score(y_test, predict5), metrics.precision_score(y_test, predict5),metrics.recall_score(y_test, predict5),metrics.f1_score(y_test, predict5)])

In [ ]:
plt.plot(hist.history['loss'], label = 'train')
plt.plot(hist.history['val_loss'], label = 'valid')
plt.legend()
plt.show()

# Conclusion

## Ploting the results

In [ ]:
#For plotting the results code is copied from Udacity Finding Donors for Charity ML Project


import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

objects = ['Accuracy score', 'Precision score', 'Recall score', 'F1 score']
w =  0.15
y_pos = np.arange(len(objects))

color = ['#A00000','#00A0A0','#00A000', '#FFFF00']
s = 0
for i in range(len(score_p)):
    plt.bar(y_pos+s, score_p[i], color = color[i], width = w )
    s += w
    
plt.xticks(y_pos + 0.22, objects)
plt.ylabel('Scores')
plt.title('Performance')
plt.suptitle("Performance Metrics for Models", fontsize = 16, y = 1.10)

p = ['SVM',  'Logistic Regression', 'Neural Network', 'LSTM']

patches = []

for k in range(len(p)):
    patches.append(mpatches.Patch(color = color[k], label = p[k]))

plt.legend(handles = patches, bbox_to_anchor = (0.5, 1.19),loc = 'upper center', borderaxespad = 0., ncol = 4, fontsize = 'x-large')

plt.show();